In [21]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from keras import backend as K
from keras.layers import Input,Lambda,Conv2D
from keras.models import load_model,Model
from keras.utils.vis_utils import plot_model,model_to_dot
from deeplearning.yolo.yad2k.models.keras_yolo import yolo_boxes_to_corners
from IPython.display import SVG

SyntaxError: invalid syntax (<ipython-input-21-d2210367f58e>, line 15)

In [3]:
#  阈值过滤，滤掉一部分box
def yolo_filter_boxes(box_confidence, boxes,box_class_probs, threshold = .6):
    box_scores = np.multiply(box_confidence,box_class_probs)
    box_classes = K.argmax(box_scores)
    box_class_scores = K.max(box_scores,axis=-1)
    filtering_mask= box_class_scores >= threshold
    scores = tf.boolean_mask(box_class_scores,filtering_mask)
    boxes  = tf.boolean_mask(boxes,filtering_mask)
    classes  = tf.boolean_mask(box_classes,filtering_mask)
    return scores,boxes,classes

In [5]:
with tf.Session() as test_a:
    box_confidence = tf.random_normal([19,19,5,1],mean=1,stddev=4,seed=1)
    boxes = tf.random_normal([19,19,5,4,],mean=1,stddev=4,seed=1)
    box_class_probs = tf.random_normal([19,19,5,80],mean=1,stddev=4,seed=1)
    scores,boxes,classes = yolo_filter_boxes(box_confidence,boxes,box_class_probs,threshold=0.5)
    print(str(scores[2].eval()))
    print(str(boxes[2].eval()))
    print(str(classes[2].eval()))
    print(str(scores.shape))
    print(str(boxes.shape))
    print(str(classes.shape))

10.750582
[ 8.426533   3.2713668 -0.5313436 -4.9413733]
7
(?,)
(?, 4)
(?,)


In [15]:
# 交并比
def iou(box1,box2):
    xi1 = np.maximum(box1[0],box2[0])
    yi1 = np.maximum(box1[1],box2[1])
    xi2 = np.minimum(box1[2],box2[2])
    yi2 = np.minimum(box1[3],box2[3])
    inter_area = (xi2 - xi1) * (yi2 - yi1)
    
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area =  box1_area + box2_area - inter_area
    iou = inter_area / union_area
    return iou

In [16]:
box1 = (2,1,4,3)
box2 = (1,2,3,4)

print("iou = ",str(iou(box1, box2)))

iou =  0.14285714285714285


In [19]:
# 非最大值抑制
def yolo_non_max_suppression(scores,boxes,classes,max_boxes = 10,iou_threshold = 0.5):
    max_boxes_tensor = K.variable(max_boxes,dtype="int32")
    K.get_session().run(tf.variables_initializer([max_boxes_tensor]))
    
    nms_indices = tf.image.non_max_suppression(boxes,scores,max_boxes,iou_threshold)
    scores = K.gather(scores,nms_indices)
    boxes = K.gather(boxes,nms_indices)
    classes = K.gather(classes,nms_indices)
    
    return scores,boxes,classes

In [20]:
with tf.Session() as test_b:
    scores = tf.random_normal([54,],mean=1,stddev=4,seed=1)
    boxes = tf.random_normal([54,4,],mean=1,stddev=4,seed=1)
    classes = tf.random_normal([54,],mean=1,stddev=4,seed=1)
    scores,boxes,classes = yolo_non_max_suppression(scores,boxes,classes)
    print(str(scores[2].eval()))
    print(str(boxes[2].eval()))
    print(str(classes[2].eval()))
    print(str(scores.shape))
    print(str(boxes.shape))
    print(str(classes.shape))

6.938395
[-5.299932    3.1379814   4.450367    0.95942086]
-2.2452729
(?,)
(?, 4)
(?,)


In [ ]:
def yolo_eval(yolo_outputs,image_shape = (720.,1280.),max_boxes = 10,score_threshold = .6,iou_threshold = .5):
    box_confidence,box_xy,box_wh,box_class_probs = yolo_outputs
    boxes  = yolo_boxes_to_corners(box_xy,box_wh)
    scores,boxes,classes = yolo_filter_boxes(box_confidence,boxes,box_class_probs,score_threshold)
    boxes = scale_boxes(boxes,image_shape)
    scores,boxes,classes = yolo_non_max_suppression(scores,boxes,classes,max_boxes,iou_threshold)
    return scores,boxes,classes